In [195]:
import openai
import dotenv
import os
import json
from typing import Any, Callable

if not dotenv.load_dotenv("../../.env"):
    raise ValueError("EnvironmentError: Failed to load `.env`")

api_key = os.getenv("OPENAI_API_KEY") or ""

if not api_key:
    raise ValueError("EnvironmentError: Failed to load `OPENAI_API_KEY`")

openai.api_key = api_key


In [196]:
def get_current_weather(location: str, unit: str = "celsius"):
    """
    Get the current weather in a given location.

    Parameters:
    location (str): The city and state, e.g. San Francisco, CA
    unit (str): The unit of temperature, can be either 'celsius' or 'fahrenheit'. Default is 'celsius'.

    Returns:
    str: A string that describes the current weather.
    """

    # This is a mock function, so let's return a mock weather report.
    weather_report = f"The current weather in {location} is 20 degrees {unit}."
    return weather_report


In [197]:
function_factory = {
    "get_current_weather": get_current_weather,
}


In [198]:
function_definitions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [199]:
def get_function(
    message: dict[str, str], functions: dict[str, Callable]
) -> dict[str, Any]:
    # Note: the JSON response may not always be valid; be sure to handle errors
    name = message["function_call"]["name"]
    callback = functions[name]
    arguments = json.loads(message["function_call"]["arguments"])
    content = callback(**arguments)
    response = {
        "role": "function",
        "name": name,
        "content": content,
    }
    return response

In [200]:
def initialize_session() -> list[dict[str, str]]:
    system_prompt = {
        "role": "system",
        "content": "My name is ChatGPT. I am a helpful assistant.",
    }
    user_input = {
        "role": "user",
        "content": "Hello! What is your name?",
    }
    messages = [system_prompt, user_input]
    return messages

In [201]:
def add_message(role: str, content: str) -> dict[str, str]:
    message = {"role": role, "content": content}
    messages.append(message)
    return message


In [202]:
def add_completion(
    response: dict[str, Any], messages: list[dict[str, str]]
) -> dict[str, str]:
    message = response["choices"][0]["message"]
    messages.append(message)
    return message

In [203]:
def print_completion(messages: list[dict[str, str]]) -> None:
    for message in messages:
        print("keys:", message.keys())
        print("role:", message["role"])
        if message["role"] in ["system", "user", "assistant"]:
            print("content:", message["content"])
            function_call = message.get("function_call", None)
            if function_call:
                print("function_call:", function_call)
        else:
            print("name:", message["name"])
            print("content:", json.dumps(message["content"]))
        print()


In [204]:
messages = initialize_session()
print_completion(messages)


keys: dict_keys(['role', 'content'])
role: system
content: My name is ChatGPT. I am a helpful assistant.

keys: dict_keys(['role', 'content'])
role: user
content: Hello! What is your name?



In [205]:
get_chat_completion = openai.ChatCompletion.create
response = get_chat_completion(model="gpt-3.5-turbo", messages=messages, temperature=0)
print(response)


{
  "id": "chatcmpl-87bRITHJ8d4Vx6X8RGFX8llf6QcGt",
  "object": "chat.completion",
  "created": 1696822904,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! My name is ChatGPT. How can I assist you today?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 31,
    "completion_tokens": 16,
    "total_tokens": 47
  }
}


In [206]:
message = add_completion(response, messages)
print_completion(messages)


keys: dict_keys(['role', 'content'])
role: system
content: My name is ChatGPT. I am a helpful assistant.

keys: dict_keys(['role', 'content'])
role: user
content: Hello! What is your name?

keys: dict_keys(['role', 'content'])
role: assistant
content: Hello! My name is ChatGPT. How can I assist you today?



In [207]:
message = add_message(
    "user", "What is the weather like today in New York City, New York?"
)
print_completion(messages)

keys: dict_keys(['role', 'content'])
role: system
content: My name is ChatGPT. I am a helpful assistant.

keys: dict_keys(['role', 'content'])
role: user
content: Hello! What is your name?

keys: dict_keys(['role', 'content'])
role: assistant
content: Hello! My name is ChatGPT. How can I assist you today?

keys: dict_keys(['role', 'content'])
role: user
content: What is the weather like today in New York City, New York?



In [208]:
response = get_chat_completion(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
)

print(response)


{
  "id": "chatcmpl-87bRKWW823bhsr6OcrxM2UyCFxPAV",
  "object": "chat.completion",
  "created": 1696822906,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"New York City, NY\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 132,
    "completion_tokens": 20,
    "total_tokens": 152
  }
}


In [209]:
message = add_completion(response, messages)
print_completion(messages)


keys: dict_keys(['role', 'content'])
role: system
content: My name is ChatGPT. I am a helpful assistant.

keys: dict_keys(['role', 'content'])
role: user
content: Hello! What is your name?

keys: dict_keys(['role', 'content'])
role: assistant
content: Hello! My name is ChatGPT. How can I assist you today?

keys: dict_keys(['role', 'content'])
role: user
content: What is the weather like today in New York City, New York?

keys: dict_keys(['role', 'content', 'function_call'])
role: assistant
content: None
function_call: {
  "name": "get_current_weather",
  "arguments": "{\n  \"location\": \"New York City, NY\"\n}"
}



In [210]:
print(message)


{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"New York City, NY\"\n}"
  }
}


In [211]:
function = get_function(message, function_factory)
print(function)


{'role': 'function', 'name': 'get_current_weather', 'content': 'The current weather in New York City, NY is 20 degrees celsius.'}


In [212]:
messages.append(function)


In [213]:
print_completion(messages)


keys: dict_keys(['role', 'content'])
role: system
content: My name is ChatGPT. I am a helpful assistant.

keys: dict_keys(['role', 'content'])
role: user
content: Hello! What is your name?

keys: dict_keys(['role', 'content'])
role: assistant
content: Hello! My name is ChatGPT. How can I assist you today?

keys: dict_keys(['role', 'content'])
role: user
content: What is the weather like today in New York City, New York?

keys: dict_keys(['role', 'content', 'function_call'])
role: assistant
content: None
function_call: {
  "name": "get_current_weather",
  "arguments": "{\n  \"location\": \"New York City, NY\"\n}"
}

keys: dict_keys(['role', 'name', 'content'])
role: function
name: get_current_weather
content: "The current weather in New York City, NY is 20 degrees celsius."



In [214]:
response = get_chat_completion(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
)

print(response)


{
  "id": "chatcmpl-87bRNXk3PYemijr6bjhVrlF9pYqMD",
  "object": "chat.completion",
  "created": 1696822909,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The current weather in New York City, New York is 20 degrees Celsius."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 177,
    "completion_tokens": 17,
    "total_tokens": 194
  }
}


In [215]:
add_completion(response, messages)


<OpenAIObject at 0x7f57141da270> JSON: {
  "role": "assistant",
  "content": "The current weather in New York City, New York is 20 degrees Celsius."
}

In [216]:
print_completion(messages)


keys: dict_keys(['role', 'content'])
role: system
content: My name is ChatGPT. I am a helpful assistant.

keys: dict_keys(['role', 'content'])
role: user
content: Hello! What is your name?

keys: dict_keys(['role', 'content'])
role: assistant
content: Hello! My name is ChatGPT. How can I assist you today?

keys: dict_keys(['role', 'content'])
role: user
content: What is the weather like today in New York City, New York?

keys: dict_keys(['role', 'content', 'function_call'])
role: assistant
content: None
function_call: {
  "name": "get_current_weather",
  "arguments": "{\n  \"location\": \"New York City, NY\"\n}"
}

keys: dict_keys(['role', 'name', 'content'])
role: function
name: get_current_weather
content: "The current weather in New York City, NY is 20 degrees celsius."

keys: dict_keys(['role', 'content'])
role: assistant
content: The current weather in New York City, New York is 20 degrees Celsius.



In [217]:
add_message("user", "Thank you! How should I dress for that kind of temperature?")
print_completion(messages)


keys: dict_keys(['role', 'content'])
role: system
content: My name is ChatGPT. I am a helpful assistant.

keys: dict_keys(['role', 'content'])
role: user
content: Hello! What is your name?

keys: dict_keys(['role', 'content'])
role: assistant
content: Hello! My name is ChatGPT. How can I assist you today?

keys: dict_keys(['role', 'content'])
role: user
content: What is the weather like today in New York City, New York?

keys: dict_keys(['role', 'content', 'function_call'])
role: assistant
content: None
function_call: {
  "name": "get_current_weather",
  "arguments": "{\n  \"location\": \"New York City, NY\"\n}"
}

keys: dict_keys(['role', 'name', 'content'])
role: function
name: get_current_weather
content: "The current weather in New York City, NY is 20 degrees celsius."

keys: dict_keys(['role', 'content'])
role: assistant
content: The current weather in New York City, New York is 20 degrees Celsius.

keys: dict_keys(['role', 'content'])
role: user
content: Thank you! How should I d

In [218]:
# Call the model again with the updated messages
response = get_chat_completion(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
)

print(response)


{
  "id": "chatcmpl-87bRPl4WjyxuySv9rtDg0sX0pC6vI",
  "object": "chat.completion",
  "created": 1696822911,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "For a temperature of 20 degrees Celsius in New York City, it is generally recommended to dress in light to medium layers. You could consider wearing a light sweater or a long-sleeved shirt with a light jacket or cardigan. Additionally, you may want to wear comfortable pants or jeans. It's also a good idea to have a pair of closed-toe shoes or sneakers. Don't forget to bring an umbrella if there is a chance of rain."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 214,
    "completion_tokens": 93,
    "total_tokens": 307
  }
}


In [219]:
add_completion(response, messages)

<OpenAIObject at 0x7f56eae23d10> JSON: {
  "role": "assistant",
  "content": "For a temperature of 20 degrees Celsius in New York City, it is generally recommended to dress in light to medium layers. You could consider wearing a light sweater or a long-sleeved shirt with a light jacket or cardigan. Additionally, you may want to wear comfortable pants or jeans. It's also a good idea to have a pair of closed-toe shoes or sneakers. Don't forget to bring an umbrella if there is a chance of rain."
}

In [220]:
print_completion(messages)

keys: dict_keys(['role', 'content'])
role: system
content: My name is ChatGPT. I am a helpful assistant.

keys: dict_keys(['role', 'content'])
role: user
content: Hello! What is your name?

keys: dict_keys(['role', 'content'])
role: assistant
content: Hello! My name is ChatGPT. How can I assist you today?

keys: dict_keys(['role', 'content'])
role: user
content: What is the weather like today in New York City, New York?

keys: dict_keys(['role', 'content', 'function_call'])
role: assistant
content: None
function_call: {
  "name": "get_current_weather",
  "arguments": "{\n  \"location\": \"New York City, NY\"\n}"
}

keys: dict_keys(['role', 'name', 'content'])
role: function
name: get_current_weather
content: "The current weather in New York City, NY is 20 degrees celsius."

keys: dict_keys(['role', 'content'])
role: assistant
content: The current weather in New York City, New York is 20 degrees Celsius.

keys: dict_keys(['role', 'content'])
role: user
content: Thank you! How should I d